In [115]:
import pandas as pd
import numpy as np

In [116]:
movies_df = pd.read_csv('movies.csv',usecols=['movieId','title'],dtype={'movieId':'int32','title':'str'})

In [117]:
rating_df = pd.read_csv('ratings.csv',usecols=['userId','movieId','rating'],dtype={'userId':'int32','movieId':'int32','rating':'float32'})

In [118]:
rating_df.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [129]:
df = pd.merge(rating_df,movies_df,on='movieId')

In [130]:
df.head()
df.shape

(100836, 4)

In [131]:
movie_rating = df.groupby('title')['userId'].count().reset_index().rename(columns={'userId':'totalratingcount'})

In [132]:
merged_df = pd.merge(df,movie_rating,on='title',how='left')

In [133]:
merged_df.head()

,userId,movieId,rating,title,totalratingcount
0,1,1,4.0,Toy Story (1995),215
1,5,1,4.0,Toy Story (1995),215
2,7,1,4.5,Toy Story (1995),215
3,15,1,2.5,Toy Story (1995),215
4,17,1,4.5,Toy Story (1995),215


In [134]:
merged_df['totalratingcount'].describe()

count    100836.000000
mean         58.758777
std          61.965384
min           1.000000
25%          13.000000
50%          39.000000
75%          84.000000
max         329.000000
Name: totalratingcount, dtype: float64

In [139]:
merged_df = merged_df[merged_df['totalratingcount']>200]

In [140]:
movie_feature_df = pd.pivot_table(index='title',columns='userId',values='rating',data=merged_df).fillna(0)

In [141]:
from scipy.sparse import csr_matrix
movie_feature_df_matrix = csr_matrix(movie_feature_df.values)

In [142]:
from sklearn.neighbors import NearestNeighbors
model_knn = NearestNeighbors(metric='cosine',algorithm='brute')
model_knn.fit(movie_feature_df_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
         metric_params=None, n_jobs=None, n_neighbors=5, p=2, radius=1.0)

In [159]:
query_index = np.random.choice(movie_feature_df.shape[0])


In [160]:
distances,indices = model_knn.kneighbors(movie_feature_df.iloc[query_index,:].values.reshape(1,-1),n_neighbors=6)

In [161]:
distances.flatten()

array([0.        , 0.3023469 , 0.36901718, 0.40957588, 0.41363066,
       0.42193377], dtype=float32)

In [162]:
for i in range(0,len(distances.flatten())):
    if i==0:
        print('Recommendation for {0}'.format(movie_feature_df.index[query_index]))
        print('\n')
    else:
        print('{0}. {1} : {2}'.format(i,movie_feature_df.index[indices.flatten()[i]],distances.flatten()[i]))

Recommendation for Seven (a.k.a. Se7en) (1995)


1. Pulp Fiction (1994) : 0.3023468852043152
2. Silence of the Lambs, The (1991) : 0.36901718378067017
3. Braveheart (1995) : 0.409575879573822
4. Forrest Gump (1994) : 0.4136306643486023
5. Usual Suspects, The (1995) : 0.42193377017974854


'Love Liza (2002)'

0.09768283